In [14]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import polars as pl 
from datetime import datetime 
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if filename.endswith('csv'):
            print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/home-credit-credit-risk-model-stability/sample_submission.csv
/kaggle/input/home-credit-credit-risk-model-stability/feature_definitions.csv
/kaggle/input/home-credit-credit-risk-model-stability/csv_files/test/test_credit_bureau_a_2_6.csv
/kaggle/input/home-credit-credit-risk-model-stability/csv_files/test/test_credit_bureau_a_2_11.csv
/kaggle/input/home-credit-credit-risk-model-stability/csv_files/test/test_credit_bureau_a_2_0.csv
/kaggle/input/home-credit-credit-risk-model-stability/csv_files/test/test_credit_bureau_a_2_9.csv
/kaggle/input/home-credit-credit-risk-model-stability/csv_files/test/test_base.csv
/kaggle/input/home-credit-credit-risk-model-stability/csv_files/test/test_credit_bureau_a_1_1.csv
/kaggle/input/home-credit-credit-risk-model-stability/csv_files/test/test_credit_bureau_b_2.csv
/kaggle/input/home-credit-credit-risk-model-stability/csv_files/test/test_credit_bureau_a_2_10.csv
/kaggle/input/home-credit-credit-risk-model-stability/csv_files/test/test_cre

# Data Cleaning


In [15]:
# drop columns with more than 90% null data
def drop_cols(df):
    for i in df.columns:
        null_values = df[f'{i}'].is_null().mean()
    if null_values > 0.9:
        df=df.drop(f'{i}')
    return  df
 
# convert data types 
def data_types(df):
    for i in df.columns:
        if i[-1] in ('D'):
            df=df.with_columns(pl.col(i).cast(pl.Date))
        if i[-1] in ('M'):
            df= df.with_columns(pl.col(i).cast(pl.String))
        if i[-1] in ('A'):
            df= df.with_columns(pl.col(i).cast(pl.Float64))
        if i[-1] in ('P'):
            df= df.with_columns(pl.col(i).cast(pl.Int16))
    return df
        

# train_applprev
train_applprev_1_0,train_applprev_1_1,train_applprev_2 are internal files with depth 1,1,2 respectively.

In [16]:
tr_applprev_0=pl.read_csv('/kaggle/input/home-credit-credit-risk-model-stability/csv_files/train/train_applprev_1_0.csv')
tr_applprev_1=pl.read_csv('/kaggle/input/home-credit-credit-risk-model-stability/csv_files/train/train_applprev_1_1.csv')
tr_applprev_2=pl.read_csv('/kaggle/input/home-credit-credit-risk-model-stability/csv_files/train/train_applprev_2.csv')

Join works differently in pandas and polars.In pandas the joining happens on index-index or index-column and merge happens on column(s) given by us. But in polars there is no merge option but only join and "on" columns are consider from both dataframes.

In [17]:
tr_applprev = pl.concat([tr_applprev_0,tr_applprev_1])
tr_applprev=tr_applprev.join(tr_applprev_2,how='left',on=['case_id','num_group1'])

In [18]:
tr_applprev=drop_cols(tr_applprev)
tr_applprev=data_types(tr_applprev)
tr_applprev.describe()
tr_applprev.head(5)

case_id,actualdpd_943P,annuity_853A,approvaldate_319D,byoccupationinc_3656910L,cancelreason_3545846M,childnum_21L,creationdate_885D,credacc_actualbalance_314A,credacc_credlmt_575A,credacc_maxhisbal_375A,credacc_minhisbal_90A,credacc_status_367L,credacc_transactions_402L,credamount_590A,credtype_587L,currdebt_94A,dateactivated_425D,district_544M,downpmt_134A,dtlastpmt_581D,dtlastpmtallstes_3545839D,education_1138M,employedfrom_700D,familystate_726L,firstnonzeroinstldate_307D,inittransactioncode_279L,isbidproduct_390L,isdebitcard_527L,mainoccupationinc_437A,maxdpdtolerance_577P,num_group1,outstandingdebt_522A,pmtnum_8L,postype_4733339M,profession_152M,rejectreason_755M,rejectreasonclient_4145042M,revolvingaccount_394A,status_219L,tenor_203L,cacccardblochreas_147M,conts_type_509L,credacc_cards_status_52L,num_group2
i64,i16,f64,date,f64,str,f64,date,f64,f64,f64,f64,str,f64,f64,str,f64,date,str,f64,date,date,str,date,str,date,str,bool,bool,f64,i16,i64,f64,f64,str,str,str,str,f64,str,f64,str,str,str,i64
2,0,640.2,null,null,"""a55475b1""",0.0,2013-04-03,null,0.0,null,null,null,null,10000.0,"""CAL""",null,null,"""P136_108_173""",0.0,null,null,"""P97_36_170""",2010-02-15,"""SINGLE""",2013-05-04,"""CASH""",false,null,8200.0,null,0,null,24.0,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,"""D""",24.0,null,"""PRIMARY_MOBILE""",null,0
2,0,640.2,null,null,"""a55475b1""",0.0,2013-04-03,null,0.0,null,null,null,null,10000.0,"""CAL""",null,null,"""P136_108_173""",0.0,null,null,"""P97_36_170""",2010-02-15,"""SINGLE""",2013-05-04,"""CASH""",false,null,8200.0,null,0,null,24.0,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,"""D""",24.0,null,"""EMPLOYMENT_PHONE""",null,1
2,0,1682.4,null,null,"""a55475b1""",0.0,2013-04-03,null,0.0,null,null,null,null,16000.0,"""CAL""",null,null,"""P136_108_173""",0.0,null,null,"""P97_36_170""",2010-02-15,"""SINGLE""",2013-05-04,"""CASH""",false,null,8200.0,null,1,null,12.0,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,"""D""",12.0,null,"""PRIMARY_MOBILE""",null,0
2,0,1682.4,null,null,"""a55475b1""",0.0,2013-04-03,null,0.0,null,null,null,null,16000.0,"""CAL""",null,null,"""P136_108_173""",0.0,null,null,"""P97_36_170""",2010-02-15,"""SINGLE""",2013-05-04,"""CASH""",false,null,8200.0,null,1,null,12.0,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,"""D""",12.0,null,"""EMPLOYMENT_PHONE""",null,1
3,0,6140.0,null,null,"""P94_109_143""",null,2019-01-07,null,0.0,null,null,null,null,59999.8,"""CAL""",null,null,"""P131_33_167""",0.0,null,null,"""P97_36_170""",2018-05-15,"""MARRIED""",2019-02-07,"""CASH""",false,null,11000.0,null,0,null,12.0,"""a55475b1""","""a55475b1""","""P94_109_143""","""a55475b1""",null,"""D""",12.0,null,"""PHONE""",null,0


In [19]:
print([i for i in tr_applprev.columns if i[-1] in ('D')])

['approvaldate_319D', 'creationdate_885D', 'dateactivated_425D', 'dtlastpmt_581D', 'dtlastpmtallstes_3545839D', 'employedfrom_700D', 'firstnonzeroinstldate_307D']


In [20]:
# creating new columns from date columns
def cols_frm_dt(df,col1,col2,name):
    df=df.with_columns((df[f'{col1}']-df[f'{col2}']).alias(f'{name}'))
    return df

tr_applprev = cols_frm_dt(tr_applprev,'approvaldate_319D','employedfrom_700D','employed_duration')
tr_applprev = cols_frm_dt(tr_applprev,'dtlastpmtallstes_3545839D','firstnonzeroinstldate_307D','contract_duration')
tr_applprev = cols_frm_dt(tr_applprev,'dtlastpmt_581D','dtlastpmtallstes_3545839D','delayed_pmt')
tr_applprev = cols_frm_dt(tr_applprev,'approvaldate_319D','creationdate_885D','time_req_for_appr')



## Aggregation of the data-

* actualdpd_943P - max
* annuity_853A - mean
* byoccupationinc_3656910L - mean
* cancelreason_3545846M -### 76
* childnum_21L - max
* credacc_actualbalance_314A - mean
* credacc_credlmt_575A - mean
* credacc_maxhisbal_375A - mean
* credacc_minhisbal_90A - mean
* credacc_status_367L - ### 7
* credacc_transactions_402L - mean
* credamount_590A - mean
* credtype_587L - ### 4
* currdebt_94A - max
* district_544M - ### 1063
* downpmt_134A - mean
* education_1138M - max
* familystate_726L- ## 6
* inittransactioncode_279L - ## 4
* isbidproduct_390L - ## company offering other services (true) 3 (0,1,null)
* isdebitcard_527L - product of other service is debit card or not
* mainoccupationinc_437A - max
* maxdpdtolerance_577P - max
* outstandingdebt_522A - max
* pmtnum_8L- max
* postype_4733339M ## 9
* profession_152M - ## 11508
* rejectreason_755M- ## 18
* rejectreasonclient_4145042M - ## 14
* revolvingaccount_394A - mean
* status_219L -## 12
* tenor_203L - mean
* cacccardblochreas_147M - ## pivot 10
* conts_type_509L - ## pivot 10
* credacc_cards_status_52L - ## pivot 7
* employed_duration - max 
* contract_duration - max
* delayed_pmt - max
* time_req_for_appr - max

In [21]:
tr_applprev.columns

['case_id',
 'actualdpd_943P',
 'annuity_853A',
 'approvaldate_319D',
 'byoccupationinc_3656910L',
 'cancelreason_3545846M',
 'childnum_21L',
 'creationdate_885D',
 'credacc_actualbalance_314A',
 'credacc_credlmt_575A',
 'credacc_maxhisbal_375A',
 'credacc_minhisbal_90A',
 'credacc_status_367L',
 'credacc_transactions_402L',
 'credamount_590A',
 'credtype_587L',
 'currdebt_94A',
 'dateactivated_425D',
 'district_544M',
 'downpmt_134A',
 'dtlastpmt_581D',
 'dtlastpmtallstes_3545839D',
 'education_1138M',
 'employedfrom_700D',
 'familystate_726L',
 'firstnonzeroinstldate_307D',
 'inittransactioncode_279L',
 'isbidproduct_390L',
 'isdebitcard_527L',
 'mainoccupationinc_437A',
 'maxdpdtolerance_577P',
 'num_group1',
 'outstandingdebt_522A',
 'pmtnum_8L',
 'postype_4733339M',
 'profession_152M',
 'rejectreason_755M',
 'rejectreasonclient_4145042M',
 'revolvingaccount_394A',
 'status_219L',
 'tenor_203L',
 'cacccardblochreas_147M',
 'conts_type_509L',
 'credacc_cards_status_52L',
 'num_group

In [22]:
tr_applprev['credacc_cards_status_52L'].unique()

credacc_cards_status_52L
str
"""INACTIVE"""
null
"""UNCONFIRMED"""
"""ACTIVE"""
"""BLOCKED"""
"""CANCELLED"""
"""RENEWED"""


In [ ]:
import plotly.express as plt
plt.bar(tr_applprev,x='case_id',y='profession_152M')

In [ ]:
tr_applprev.filter(tr_applprev['case_id']==2703454)

In [ ]:
pl.Config(fmt_str_lengths=1000)
pl.Config.set_tbl_rows(500)
feature_definition= pl.read_csv('/kaggle/input/home-credit-credit-risk-model-stability/feature_definitions.csv')

feature_definition.head(500)#[feature_definition.Variable=='persontype_792L']
